In [1]:
import requests
import json
from tqdm.notebook import tqdm
from time import sleep
import os

def date_range(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

commit_data = {}
sleep_time = 1 * 60


def get_commit_count(date):
  date = date.strftime("%Y-%m-%d")
  url = f"https://api.github.com/search/commits?q=committer-date:{date}"
  payload={}
  headers = {
      'Authorization': os.getenv('GITHUB_AUTH_TOKEN')
  }
  response = requests.request("GET", url, headers=headers, data=payload)
  try:
      commit_count = json.loads(response.text)['total_count']
      commit_data[date] = commit_count
  except:
      print(f'{date}: {json.loads(response.text)}')


In [2]:
from datetime import date, timedelta
start_date = date(2017, 1, 1)
end_date = date(2017, 1, 2)

prog_bar = tqdm(total=365)

def prog_update():
  prog_bar.update(30)

for date in date_range(start_date, end_date):
    
  days_elapsed = int((date - start_date).days)
    
  if (days_elapsed % 30 == 0) & (days_elapsed > 0):
    prog_update()
    sleep(sleep_time)
        
  get_commit_count(date)    


prog_bar.update(5)
prog_bar.close()
print("Done!")

  0%|          | 0/365 [00:00<?, ?it/s]

Done!


In [3]:
import pandas as pd
commit_df = pd.DataFrame.from_dict(commit_data, 
                                   orient='index', 
                                   columns=['commit_count'])
commit_df.reset_index(inplace=True)
commit_df.rename(columns = {'index':'date'}, inplace=True)
commit_df

,date,commit_count
0,2017-01-01,483421


In [4]:
import os

commit_filename = "commit_data_2017_sept.csv"

try:
    os.remove(commit_filename)
except:
    print(f"{commit_filename} does not exist.")

commit_df.to_csv(commit_filename, index=False)
print(f"New {commit_filename} file created!")

commit_data_2017_sept.csv does not exist.
New commit_data_2017_sept.csv file created!
